In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.1'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

2022-10-28 15:43:23.357413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 15:43:23.363673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 15:43:23.364299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA), 
                 ("train_standardize", imageStandardize), 
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_process_a", valPreprocessA), 
               ("val_standardize", imageStandardize), 
               ("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-10-28 15:43:23.620705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 15:43:23.621407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 15:43:23.621761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 15:43:23.622060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 227, 227, 3), dtype=float32, numpy=
array([[[[-1.5244231 , -1.3165796 , -1.2936594 ],
         [-1.5244231 , -1.3165796 , -1.2936594 ],
         [-1.5244231 , -1.3165796 , -1.2936594 ],
         ...,
         [-1.1652129 , -1.0862019 , -0.99170804],
         [-1.1629463 , -1.0839355 , -0.9894417 ],
         [-1.1890334 , -1.1100225 , -1.0155287 ]],

        [[-1.5244231 , -1.3165796 , -1.2936594 ],
         [-1.5244231 , -1.3165796 , -1.2936594 ],
         [-1.5244231 , -1.3165796 , -1.2936594 ],
         ...,
         [-1.0957116 , -1.0167007 , -0.9222069 ],
         [-1.16822   , -1.0892092 , -0.99471533],
         [-1.2201179 , -1.1411071 , -1.0466132 ]],

        [[-1.5244231 , -1.3165796 , -1.2936594 ],
         [-1.5244231 , -1.3165796 , -1.2936594 ],
         [-1.5244231 , -1.3165796 , -1.2936594 ],
         ...,
         [-1.15213   , -1.073119  , -0.97862524],
         [-1.1432934 , -1.0642824 , -0.9697886 ],
         [-1.1769553 , -1.0979445 , -1.0034506

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(CROP_HEIGHT, CROP_WIDTH, 3))

    backbone = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, input_shape=(CROP_HEIGHT, CROP_WIDTH, 3))
    backbone.trainable = False    
    
    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = tf.keras.applications.resnet_v2.preprocess_input(inp)
    o = backbone(o, training=False)
    o = flat1(o)
    o = fc1(o)
    o = do1(o)

    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='baseline_1')
    
    return model

In [28]:
model = createModel()

In [29]:
model.summary()

Model: "baseline_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 227, 227, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 227, 227, 3)      0         
 a)                                                              
                                                                 
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 flat1 (Flatten)             (None, 131072)            0         
                                                                 
 fc1 (Dense)                 (None, 128)               1

### Compile Model

In [30]:
STEPS_PER_EPOCH = 1000
EPOCH = 100

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [31]:
def schedule(epoch, lr):
    return lr

In [32]:
logBasePath = "log/baseline_resnet_1"
logPrefix = "log"

In [33]:
for i, (train, val) in enumerate(folds):
    
    model = createModel()
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb, lrSchedule]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-10-28 15:43:28.208332: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-10-28 15:43:28.208356: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-10-28 15:43:28.208377: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-10-28 15:43:28.305057: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-10-28 15:43:28.306269: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-10-28 15:43:29.404229: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  19/3528 [..............................] - ETA: 31s - loss: 0.7749 - accuracy: 0.3816  

2022-10-28 15:43:30.394991: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3528/3528 [==============================] - 31s 8ms/step - loss: 0.7098 - accuracy: 0.5232
Epoch 1/100
   1/1000 [..............................] - ETA: 24:14 - loss: 0.9528 - accuracy: 0.0000e+00

2022-10-28 15:44:00.488601: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-10-28 15:44:00.488628: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.


...

2022-10-28 19:50:20.973038: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-10-28 19:50:20.997035: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-10-28 19:50:21.063838: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 3210 callback api events and 3185 activity events. 
2022-10-28 19:50:21.122360: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.


  10/1000 [..............................] - ETA: 1:39 - loss: 0.8706 - accuracy: 0.6500

2022-10-28 19:50:21.178567: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: log/baseline_resnet_1/log_4/plugins/profile/2022_10_28_19_50_21

2022-10-28 19:50:21.210959: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to log/baseline_resnet_1/log_4/plugins/profile/2022_10_28_19_50_21/rachanon-pc1.trace.json.gz
2022-10-28 19:50:21.301606: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: log/baseline_resnet_1/log_4/plugins/profile/2022_10_28_19_50_21

2022-10-28 19:50:21.307212: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to log/baseline_resnet_1/log_4/plugins/profile/2022_10_28_19_50_21/rachanon-pc1.memory_profile.json.gz
2022-10-28 19:50:21.308845: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: log/baseline_resnet_1/log_4/plugins/profile/2022_10_28_19_50_21
Dumped tool data

1000/1000 [==============================] - 40s 38ms/step - loss: 0.6487 - accuracy: 0.6385 - val_loss: 0.6664 - val_accuracy: 0.5924 - lr: 0.0010
Epoch 2/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6624 - accuracy: 0.6432 - val_loss: 0.7550 - val_accuracy: 0.4766 - lr: 0.0010
Epoch 3/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6636 - accuracy: 0.6033 - val_loss: 0.6884 - val_accuracy: 0.4775 - lr: 0.0010
Epoch 4/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6669 - accuracy: 0.6087 - val_loss: 0.7336 - val_accuracy: 0.5307 - lr: 0.0010
Epoch 5/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6927 - accuracy: 0.5670 - val_loss: 0.6899 - val_accuracy: 0.4772 - lr: 0.0010
Epoch 6/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6437 - accuracy: 0.6385 - val_loss: 0.7667 - val_accuracy: 0.4776 - lr: 0.0010
Epoch 7/100
1000/1000 [=============================

1000/1000 [==============================] - 37s 37ms/step - loss: 0.6835 - accuracy: 0.5633 - val_loss: 0.6956 - val_accuracy: 0.5233 - lr: 0.0010
Epoch 53/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6924 - accuracy: 0.5215 - val_loss: 0.6926 - val_accuracy: 0.5233 - lr: 0.0010
Epoch 54/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6954 - accuracy: 0.4790 - val_loss: 0.6969 - val_accuracy: 0.4769 - lr: 0.0010
Epoch 55/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6940 - accuracy: 0.5000 - val_loss: 0.6920 - val_accuracy: 0.5233 - lr: 0.0010
Epoch 56/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6858 - accuracy: 0.5455 - val_loss: 0.6930 - val_accuracy: 0.5233 - lr: 0.0010
Epoch 57/100
1000/1000 [==============================] - 37s 37ms/step - loss: 0.6919 - accuracy: 0.5148 - val_loss: 0.6931 - val_accuracy: 0.4767 - lr: 0.0010
Epoch 58/100
1000/1000 [=======================